In [68]:
import pandas as pd
from utils import * 
import geopandas as gpd
from main import *

pd.set_option('display.max_columns', None)

In [79]:
datapaths = {
    "Wind": "data/wind/ez_gis.plant_power_eia_v8_wind.shp",
    "GDP": "data/gdp_data.csv",
    "Solar": "data/solar_data.csv",
    "education": "data/education_data.csv",
    "private_schools": "data/private_school_data.csv",
    "DEC_race": "data/race_dec_data.csv",
    "ACS_race": "data/race_acs_data.csv",
    "election": "data/election_data.csv",
    "income": "data/income_data.csv",
    "unemployment": "data/unemployment_data.csv",
    "NREL_Electric": "data/NREL_Electric_data.csv",
    "Rural_Urban": "data/rural_urban.csv",
    'Population Data': 'data/pop_data.csv',
}
bounding_box = pd.read_csv("/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/data cleaning/data/county_bounding_boxes.csv", dtype={"FIPS State": str, "FIPS County": str})

In [81]:
FIPS = pd.read_csv("data/FIPS.csv", dtype={"FIPS State": str, "FIPS County": str})
solar = get_solar(datapaths["Solar"], bounding_box, size='all')
wind = get_wind(datapaths["Wind"], bounding_box)
edu_18_24 = get_education_18_24(datapaths["education"])
edu_25 = get_education_25_over(datapaths["education"])
priv_sch = get_no_priv_schools(datapaths["private_schools"])
race_dec = get_race_dec(datapaths["DEC_race"])
race_acs = get_race_acs(datapaths["ACS_race"])
elections = get_election(datapaths["election"])
income = get_income(datapaths["income"])
unemployment = get_unemployment(datapaths["unemployment"])
electric = NREL_Electric(datapaths["NREL_Electric"])
gdp = get_GDP(datapaths["GDP"], bounding_box, datapaths["Population Data"])
rural_urban = get_rural_urban_coverage(datapaths["Rural_Urban"])
save_datapath = "missing_counties/"

/Users/jack/Library/CloudStorage/GoogleDrive-limjackailjk@gmail.com/My Drive/Solar PV Lab/NIMBY Project/Solar NIMBY Final/Solar-NIMBY/data cleaning/utils.py:269: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,12,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,35,36,37,38,40,41,42,43,44,46,47,48,49,50,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,106,107,108,109,118,119,122,123,124,125,132,133,134,135,136,137,138,139,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,162,163,164,165,168,169,170,171,174,175,176,177,180,181,182,183,186,187,188,189,192,193,194,195,222,223,224,225,228,229,230,231,234,235,236,237,238,239,240,241,242,243,244,245,258,260,261,262,263,264,265,266,267,268,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,290,291,292,293,294,296,297,298,299,300,302,303,304,305,306,308,309,310,311,312,313,314,31

In [82]:
# put all df into a dictionary
dfs = {
    "solar": solar,
    "wind": wind,
    "edu_18_24": edu_18_24,
    "edu_25": edu_25,
    "priv_sch": priv_sch,
    "race_acs": race_acs,
    "race_dec": race_dec,
    "elections_demo": elections['democrat'],
    "elections_repub": elections['republican'],
    "elections_other": elections['other'],
    "elections_green": elections['green'],
    "elections_libert": elections['libertarian'],
    "income": income,
    "unemployment": unemployment,
    "electric": electric,
    "gdp": gdp,
    "rural_urban": rural_urban
}

In [72]:
def get_missing_counties(data, base):
    base['State_County'] = base['State'] + '_' + base['County Name']
    data['State_County'] = data['State'] + '_' + data['County Name']
    
    not_in = base[~base['State_County'].isin(data['State_County'])]
    not_in = not_in.drop(columns=['State_County'])
    return not_in

## Count the number of counties that don't have data at all for each factor

In [83]:
dfs_missing = {
    name: get_missing_counties(data, FIPS) for name, data in dfs.items()
}

for name, data in dfs_missing.items():
    data.to_csv(save_datapath + name + "_missing.csv", index=False)
    print(f"For {name}, {len(data)} counties are missing")

For solar, 2265 counties are missing
For wind, 2645 counties are missing
For edu_18_24, 5 counties are missing
For edu_25, 5 counties are missing
For priv_sch, 1048 counties are missing
For race_acs, 5 counties are missing
For race_dec, 5 counties are missing
For elections_demo, 5 counties are missing
For elections_repub, 5 counties are missing
For elections_other, 618 counties are missing
For elections_green, 1346 counties are missing
For elections_libert, 167 counties are missing
For income, 5 counties are missing
For unemployment, 5 counties are missing
For electric, 636 counties are missing
For gdp, 70 counties are missing
For rural_urban, 5 counties are missing


## Count number of NaNs in each dfs

In [84]:
counts_str = ""

for name, df in dfs.items():
    temp = df.copy()
    temp = temp.drop(columns=['State', 'County Name'])
    
    # Count rows that have 1 or more 0s
    count_zeros = temp[temp.eq(0).any(axis=1)]
    
    # Count rows that have a NaN
    count_nan = temp[temp.isna().any(axis=1)]
    
    print(f"{name}:")
    print(f"Rows with 0s: {count_zeros.shape[0]}")
    print(f"Rows with NaNs: {count_nan.shape[0]}")
    print("\n")
    
    counts_str += f"{name}:\n Rows with 0s: {count_zeros.shape[0]}\n Rows with NaNs: {count_nan.shape[0]}\n\n"

solar:
Rows with 0s: 0
Rows with NaNs: 0


wind:
Rows with 0s: 21
Rows with NaNs: 0


edu_18_24:
Rows with 0s: 53
Rows with NaNs: 0


edu_25:
Rows with 0s: 7
Rows with NaNs: 0


priv_sch:
Rows with 0s: 0
Rows with NaNs: 0


race_acs:
Rows with 0s: 1469
Rows with NaNs: 0


race_dec:
Rows with 0s: 616
Rows with NaNs: 0


elections_demo:
Rows with 0s: 0
Rows with NaNs: 0


elections_repub:
Rows with 0s: 0
Rows with NaNs: 0


elections_other:
Rows with 0s: 177
Rows with NaNs: 0


elections_green:
Rows with 0s: 84
Rows with NaNs: 0


elections_libert:
Rows with 0s: 2
Rows with NaNs: 0


income:
Rows with 0s: 0
Rows with NaNs: 0


unemployment:
Rows with 0s: 9
Rows with NaNs: 0


electric:
Rows with 0s: 10
Rows with NaNs: 0


gdp:
Rows with 0s: 0
Rows with NaNs: 0


rural_urban:
Rows with 0s: 1097
Rows with NaNs: 0


